#### 한국어 코퍼스 전처리 (*Torchtext 없이 해보기)
- 데이터셋 : Korpora에서 로딩
- 형태소분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비 <HR>

In [1]:
## 모듈 로딩
from Korpora import Korpora                         # Open Korean Dataset
from konlpy.tag import *                            # 형태소 분석기
import spacy                                        # 형태소 분석기

from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize
from torch.utils.data import Dataset, DataLoader    # Pytorch Dataset 관련 모듈
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [2]:
### 데이터 로딩
nsmc = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-50\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [3]:
# nsmc.train[0], nsmc.test[0]

In [4]:
# Test 데이터셋을 DataFrame으로 로딩
# nsmcDF = pd.DataFrame(nsmc.test)
# nsmcDF.info()

[3] 텍스트 데이터 전처리 <hr>
- 토큰화 / 정제 (불용어, 구두점, 띄어쓰기, 오타 등등 처리)
- 단어사전 생성
- 문장 ==> 수치화

In [5]:
### 형태소 분석기
# 한나눔
han = Hannanum()

# 꼬꼬마
kkma = Kkma()

# OKT
okt = Okt()

In [6]:
sample = nsmc.test.texts[:100]

In [7]:
for i in sample[:10]:
    print(i)

굳 ㅋ
GDNTOPCLASSINTHECLUB
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
지루하지는 않은데 완전 막장임... 돈주고 보기에는....
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
음악이 주가 된, 최고의 음악영화
진정한 쓰레기
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..


- 불용어 & 구두점 제거

In [8]:
import re
import string
punc = string.punctuation

In [12]:
all_text=[]
for text in sample[::-1]:
    text=text.replace('\n','')
    text=text.replace(punc,'')
    if len(text): all_text.append(text)

In [13]:
for tokenList in all_text[:10]:
    print(tokenList, end='\n')

가끔 문득 생각나서 다시보는 영화..색감이 정말 예술이죠 강렬하고 화려하고..츠지야안나의 너무도 아름답던 리즈시절도 볼 수 있고..
1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴 여자들 만나고 다니다가 결국 맥 라이언한테 꽂힌 톰 행크스나약혼자 두고 톰 행크스랑 썸 타다가 결국엔 약혼자 버리고 톰행크스한테 쪼르르 달려가는 맥 라이언이나.
제임스 완이 내 목표임 ㄷ
완전 재밌엇는데 왜 평점이??
전기톱은못들고다니는데 엔진톱이겠죠
뭘 만든 건가?
단 두마디 '감동'
엔딩때까지 음성 넣은 것은 오버의 극치다
후속작 계획은 없나요..? ㅜ
로코 굉장히 즐겨보는데, 이 영화는 좀 별로였다. 뭔가 사랑도 개그도 억지스런 느낌..


In [34]:
# makestr=",".join(nsmc.test.texts)
makestr=",".join(all_text)

In [35]:
# 불용어,구두점 제거 (정규식) 
import re
import string

# newmake = re.sub('[\,.? ]', '', makestr)

In [36]:
# 형태소 분석

# han_result=han.morphs(makestr)
# kkma_result=kkma.morphs(makestr)
okt_result=okt.morphs(makestr)
# okt_result=okt.morphs(newmake)

In [37]:
all_tokens=[]
for text in okt_result:
    all_tokens.append( wordpunct_tokenize(text) )

In [38]:
len(all_tokens)

1378

In [39]:
# 한국어 불용어 제거
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

In [40]:
STOP_PATH = '../Data/stopwords-ko.txt'

In [41]:
with open(STOP_PATH, 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

In [42]:
mytoken = remove_stopwords(all_tokens)

In [43]:
for tokenList in mytoken:
    for token in tokenList:
        if (token in punc) & (token == 'ㅋ'):
            tokenList.remove(token)

In [44]:
for tokenList in mytoken:
    print(tokenList)

['가끔']
['문득']
['생각나서']
['다시']
['보는']
['영화']
['..']
['색감']
['이']
['정말']
['예술']
['이']
['죠']
['강렬하고']
['화려하고']
['..']
['츠지']
['야']
['안나']
['의']
['너무도']
['아름답던']
['리즈시절']
['도']
['볼']
['수']
['있고']
['..,']
['1']
['점']
['고']
['아깝다']
['.']
['개막']
['장']
['영화']
['의']
['원조']
['라고']
['나']
['할까']
['.']
['아내']
['와']
['사별']
['한']
['지']
['얼마나']
['지났다고']
['딴']
['여자']
['들']
['만나고']
['다니다가']
['결국']
['맥']
['라이언']
['한테']
['꽂힌']
['톰']
['행크스']
['나']
['약혼자']
['두고']
['톰']
['행크스']
['랑']
['썸']
['타']
['다가']
['결국']
['엔']
['약혼자']
['버리고']
['톰']
['행크스']
['한테']
['쪼르르']
['달려가는']
['맥']
['라이언']
['이나']
['.,']
['제임스']
['완']
['이']
['내']
['목표']
['임']
['ㄷ']
[',']
['완전']
['재밌']
['엇']
['는데']
['왜']
['평점']
['이']
['??,']
['전기톱']
['은']
['못']
['들고다니는데']
['엔진']
['톱']
['이']
['겠죠']
[',']
['뭘']
['만든']
['건가']
['?,']
['단']
['두']
['마디']
["'"]
['감동']
["',"]
['엔딩']
['때']
['까지']
['음성']
['넣은']
['것']
['은']
['오버']
['의']
['극치']
['다']
[',']
['후속작']
['계획']
['은']
['없나요']
['..?']
['ㅜ']
[',']
['로코']
['굉장히']
['즐겨']
['보는데']
[',']
['이']
['영화']
['는']
['좀'

- 단어별 빈도수 계산

In [45]:
# 토큰을 키로 해서 빈도수 저장
token_freg={} # dict에서는 key값을 찾아봄

# 라인(줄)별 토큰을 읽어서 빈도 체크
for tokenList in mytoken:
    for token in tokenList:
        # 카운트시, 토큰 key가 존재하지 않으면 key 추가
        if token not in token_freg:
            token_freg[token] = 1
        # 카운트시, 이미 존재하는 토큰 key는 1 증가
        else:
            token_freg[token] += 1

In [46]:
token_freg

{'가끔': 1,
 '문득': 1,
 '생각나서': 1,
 '다시': 1,
 '보는': 1,
 '영화': 24,
 '..': 12,
 '색감': 1,
 '이': 34,
 '정말': 2,
 '예술': 1,
 '죠': 2,
 '강렬하고': 1,
 '화려하고': 1,
 '츠지': 1,
 '야': 4,
 '안나': 1,
 '의': 25,
 '너무도': 1,
 '아름답던': 1,
 '리즈시절': 1,
 '도': 13,
 '볼': 3,
 '수': 2,
 '있고': 1,
 '..,': 4,
 '1': 3,
 '점': 3,
 '고': 3,
 '아깝다': 1,
 '.': 18,
 '개막': 2,
 '장': 2,
 '원조': 1,
 '라고': 3,
 '나': 3,
 '할까': 1,
 '아내': 1,
 '와': 4,
 '사별': 1,
 '한': 4,
 '지': 2,
 '얼마나': 1,
 '지났다고': 1,
 '딴': 1,
 '여자': 1,
 '들': 10,
 '만나고': 1,
 '다니다가': 1,
 '결국': 3,
 '맥': 2,
 '라이언': 2,
 '한테': 3,
 '꽂힌': 1,
 '톰': 3,
 '행크스': 3,
 '약혼자': 2,
 '두고': 2,
 '랑': 1,
 '썸': 1,
 '타': 1,
 '다가': 1,
 '엔': 1,
 '버리고': 1,
 '쪼르르': 1,
 '달려가는': 1,
 '이나': 2,
 '.,': 16,
 '제임스': 2,
 '완': 1,
 '내': 5,
 '목표': 1,
 '임': 2,
 'ㄷ': 1,
 ',': 62,
 '완전': 4,
 '재밌': 1,
 '엇': 1,
 '는데': 1,
 '왜': 2,
 '평점': 9,
 '??,': 3,
 '전기톱': 1,
 '은': 15,
 '못': 6,
 '들고다니는데': 1,
 '엔진': 1,
 '톱': 1,
 '겠죠': 1,
 '뭘': 1,
 '만든': 1,
 '건가': 1,
 '?,': 3,
 '단': 1,
 '두': 2,
 '마디': 1,
 "'": 3,
 '감동': 2,
 "',": 2,
 '엔딩'

In [47]:
# 단어들 별로 빈도수 저장
freqsDict={}
for k, v in token_freg.items():
    if v not in freqsDict:
        freqsDict[v]=[k]
    else:
        freqsDict[v].append(k)
print(freqsDict)

{1: ['가끔', '문득', '생각나서', '다시', '보는', '색감', '예술', '강렬하고', '화려하고', '츠지', '안나', '너무도', '아름답던', '리즈시절', '있고', '아깝다', '원조', '할까', '아내', '사별', '얼마나', '지났다고', '딴', '여자', '만나고', '다니다가', '꽂힌', '랑', '썸', '타', '다가', '엔', '버리고', '쪼르르', '달려가는', '완', '목표', 'ㄷ', '재밌', '엇', '는데', '전기톱', '들고다니는데', '엔진', '톱', '겠죠', '뭘', '만든', '건가', '단', '마디', '엔딩', '음성', '넣은', '오버', '극치', '후속작', '계획', '없나요', '..?', 'ㅜ', '로코', '굉장히', '즐겨', '보는데', '별로', '뭔가', '개그', '억지스런', '느낌', '바다', '4', '차원', '고양이', '도라에몽', '하는', '진구', '친구', '미지', '탐험', '이야기', '우주', '꺼', '갔음', '좋겠당', '하는데', '단어', '필요없다', '재수없는', '충분하다', '중간', '화면', '끊기는것', '빼곤', '좋았어요', '재미있었다', '봐야', 'ㅎ', '아찔한', '줄다리기', '1956년', '믿기지가', '않을', '디테', '일하다', '안소니', '퀸', '명연기', '또한', '물론', '제목', '찰지구나', '개성', '있는', '몬스터', '귀여운', '여자아이', '만나', '뿜어내는', '매력', '퐁당', '빠졌다', '솔직히', 'C', '급', '이하', '이긴', 'ㅎㅎ', '제시카', '벗고', '달려드는데', '쌩까는게', '되냐', '대단한것', '같습니다', '천', '재', '..?,', '쇼타', '군', '돈키호테', '대박', '재밌음', 'ㅠㅜㅜㅜ', '없을', '하', '작은', '그림', '기대했는데', '수염', '색깔', '원작', '비', '충실', 

In [48]:
# 단어등장 횟수, 단어 개수 및 단어이름 별로 빈도수 저장
freqsDict={}
for k, v in token_freg.items():
    if v not in freqsDict:
        # freqsDict[v]=[k]
        # freqsDict[v]=1
        freqsDict[v]=[1, [k]]
    else:
        # freqsDict[v].append(k)
        # freqsDict[v]+=1
        freqsDict[v][0]+=1
        freqsDict[v][1].append(k)
print(freqsDict)

{1: [639, ['가끔', '문득', '생각나서', '다시', '보는', '색감', '예술', '강렬하고', '화려하고', '츠지', '안나', '너무도', '아름답던', '리즈시절', '있고', '아깝다', '원조', '할까', '아내', '사별', '얼마나', '지났다고', '딴', '여자', '만나고', '다니다가', '꽂힌', '랑', '썸', '타', '다가', '엔', '버리고', '쪼르르', '달려가는', '완', '목표', 'ㄷ', '재밌', '엇', '는데', '전기톱', '들고다니는데', '엔진', '톱', '겠죠', '뭘', '만든', '건가', '단', '마디', '엔딩', '음성', '넣은', '오버', '극치', '후속작', '계획', '없나요', '..?', 'ㅜ', '로코', '굉장히', '즐겨', '보는데', '별로', '뭔가', '개그', '억지스런', '느낌', '바다', '4', '차원', '고양이', '도라에몽', '하는', '진구', '친구', '미지', '탐험', '이야기', '우주', '꺼', '갔음', '좋겠당', '하는데', '단어', '필요없다', '재수없는', '충분하다', '중간', '화면', '끊기는것', '빼곤', '좋았어요', '재미있었다', '봐야', 'ㅎ', '아찔한', '줄다리기', '1956년', '믿기지가', '않을', '디테', '일하다', '안소니', '퀸', '명연기', '또한', '물론', '제목', '찰지구나', '개성', '있는', '몬스터', '귀여운', '여자아이', '만나', '뿜어내는', '매력', '퐁당', '빠졌다', '솔직히', 'C', '급', '이하', '이긴', 'ㅎㅎ', '제시카', '벗고', '달려드는데', '쌩까는게', '되냐', '대단한것', '같습니다', '천', '재', '..?,', '쇼타', '군', '돈키호테', '대박', '재밌음', 'ㅠㅜㅜㅜ', '없을', '하', '작은', '그림', '기대했는데', '수염', '색깔', '원작', '비', 

In [49]:
# [(단어나온 횟수, [단어개수, [단어이름]]) ...]
sorted(freqsDict.items(), reverse=True)

[(62, [1, [',']]),
 (34, [1, ['이']]),
 (25, [1, ['의']]),
 (24, [1, ['영화']]),
 (18, [1, ['.']]),
 (16, [1, ['.,']]),
 (15, [2, ['은', '는']]),
 (13, [1, ['도']]),
 (12, [2, ['..', '가']]),
 (10, [3, ['들', '에', '을']]),
 (9, [2, ['평점', '를']]),
 (7, [3, ['그', '뭐', '에서']]),
 (6, [4, ['못', '!', '게', '이다']]),
 (5, [7, ['내', '말', '로', '이건', '만', '...', '진짜']]),
 (4,
  [21,
   ['야',
    '..,',
    '와',
    '한',
    '완전',
    '때',
    '최고',
    '이런',
    '관객모독',
    '봐도',
    '없는',
    '놈',
    '모든',
    '너무',
    '재밌다',
    '시간',
    '아',
    '적',
    '...,',
    '같은',
    '저']]),
 (3,
  [37,
   ['볼',
    '1',
    '점',
    '고',
    '라고',
    '나',
    '결국',
    '한테',
    '톰',
    '행크스',
    '??,',
    '?,',
    "'",
    '까지',
    '것',
    '좀',
    '~,',
    '면',
    '내용',
    '좋다',
    '재미',
    '!,',
    '보고',
    '빵점',
    '하고',
    '잘',
    '연기',
    '감독',
    '더',
    '....',
    '한다',
    '이영화',
    '음악',
    '개',
    '별',
    '시리즈',
    '3']]),
 (2,
  [86,
   ['정말',
    '죠',
    '수',
    '개막',

In [50]:
## 빈도가 높게 나오는 순서대로 단어 정렬
#                k, v -----------------------------|
storedTokens=sorted(token_freg.items(), reverse=True, key=lambda x:x[1])

In [51]:
## 단어사전 생성 및 초기화
## 특수토큰 : 'PAD', 'OOV' 또는 'UNK'
PAD_TOKEN, OOV_TOKEN='PAD', 'OOV'
vocab={PAD_TOKEN:0, OOV_TOKEN:1}

In [52]:
## 데이터 코퍼스에서 추출된 토큰(단어)들
# vocab[]

for idx, tk in enumerate(storedTokens, 2):
    vocab[tk[0]] = idx
print(vocab)

{'PAD': 0, 'OOV': 1, ',': 2, '이': 3, '의': 4, '영화': 5, '.': 6, '.,': 7, '은': 8, '는': 9, '도': 10, '..': 11, '가': 12, '들': 13, '에': 14, '을': 15, '평점': 16, '를': 17, '그': 18, '뭐': 19, '에서': 20, '못': 21, '!': 22, '게': 23, '이다': 24, '내': 25, '말': 26, '로': 27, '이건': 28, '만': 29, '...': 30, '진짜': 31, '야': 32, '..,': 33, '와': 34, '한': 35, '완전': 36, '때': 37, '최고': 38, '이런': 39, '관객모독': 40, '봐도': 41, '없는': 42, '놈': 43, '모든': 44, '너무': 45, '재밌다': 46, '시간': 47, '아': 48, '적': 49, '...,': 50, '같은': 51, '저': 52, '볼': 53, '1': 54, '점': 55, '고': 56, '라고': 57, '나': 58, '결국': 59, '한테': 60, '톰': 61, '행크스': 62, '??,': 63, '?,': 64, "'": 65, '까지': 66, '것': 67, '좀': 68, '~,': 69, '면': 70, '내용': 71, '좋다': 72, '재미': 73, '!,': 74, '보고': 75, '빵점': 76, '하고': 77, '잘': 78, '연기': 79, '감독': 80, '더': 81, '....': 82, '한다': 83, '이영화': 84, '음악': 85, '개': 86, '별': 87, '시리즈': 88, '3': 89, '정말': 90, '죠': 91, '수': 92, '개막': 93, '장': 94, '지': 95, '맥': 96, '라이언': 97, '약혼자': 98, '두고': 99, '이나': 100, '제임스': 101, '임': 102, '왜': 103

In [53]:
## 텍스트 문장 ===> 수치화 [인코딩]
encodingData=[]
for tokenList in all_tokens:
    sent=[]
    print(f'문장: {tokenList}')
    for token in tokenList:
        sent.append(vocab[token])
    
    # 인코딩 된 문장 저장
    encodingData.append(sent)
    print(f'==>인코딩: {sent}\n')

문장: ['가끔']
==>인코딩: [176]

문장: ['문득']
==>인코딩: [177]

문장: ['생각나서']
==>인코딩: [178]

문장: ['다시']
==>인코딩: [179]

문장: ['보는']
==>인코딩: [180]

문장: ['영화']
==>인코딩: [5]

문장: ['..']
==>인코딩: [11]

문장: ['색감']
==>인코딩: [181]

문장: ['이']
==>인코딩: [3]

문장: ['정말']
==>인코딩: [90]

문장: ['예술']
==>인코딩: [182]

문장: ['이']
==>인코딩: [3]

문장: ['죠']
==>인코딩: [91]

문장: ['강렬하고']
==>인코딩: [183]

문장: ['화려하고']
==>인코딩: [184]

문장: ['..']
==>인코딩: [11]

문장: ['츠지']
==>인코딩: [185]

문장: ['야']
==>인코딩: [32]

문장: ['안나']
==>인코딩: [186]

문장: ['의']
==>인코딩: [4]

문장: ['너무도']
==>인코딩: [187]

문장: ['아름답던']
==>인코딩: [188]

문장: ['리즈시절']
==>인코딩: [189]

문장: ['도']
==>인코딩: [10]

문장: ['볼']
==>인코딩: [53]

문장: ['수']
==>인코딩: [92]

문장: ['있고']
==>인코딩: [190]

문장: ['..,']
==>인코딩: [33]

문장: ['1']
==>인코딩: [54]

문장: ['점']
==>인코딩: [55]

문장: ['고']
==>인코딩: [56]

문장: ['아깝다']
==>인코딩: [191]

문장: ['.']
==>인코딩: [6]

문장: ['개막']
==>인코딩: [93]

문장: ['장']
==>인코딩: [94]

문장: ['영화']
==>인코딩: [5]

문장: ['의']
==>인코딩: [4]

문장: ['원조']
==>인코딩: [192]

문장: ['라고']
==>인코딩: [57]

문장: ['나']
==>인코딩